# Advent of Code 2020 Day 10
[link](https://adventofcode.com/2020/day/10)

We shall call the $n$-jolt difference as $n$-diff for short.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#dependencies" data-toc-modified-id="dependencies-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>dependencies</a></span></li><li><span><a href="#read-input" data-toc-modified-id="read-input-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>read input</a></span></li><li><span><a href="#part-1" data-toc-modified-id="part-1-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>part 1</a></span><ul class="toc-item"><li><span><a href="#answer" data-toc-modified-id="answer-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>answer</a></span></li></ul></li><li><span><a href="#part-2" data-toc-modified-id="part-2-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>part 2</a></span><ul class="toc-item"><li><span><a href="#solution-1" data-toc-modified-id="solution-1-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>solution 1</a></span></li><li><span><a href="#solution-2" data-toc-modified-id="solution-2-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>solution 2</a></span></li><li><span><a href="#solution-3" data-toc-modified-id="solution-3-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>solution 3</a></span></li><li><span><a href="#answer" data-toc-modified-id="answer-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>answer</a></span></li></ul></li></ul></div>

## dependencies

## read input

In [1]:
function parse_input(filename)
    result = parse.(Int, readlines(filename))
end

parse_input (generic function with 1 method)

In [2]:
input_sample_1 = parse_input("input_sample_1.txt")

11-element Array{Int64,1}:
 16
 10
 15
  5
  1
 11
  7
 19
  6
 12
  4

In [3]:
input_sample_2 = parse_input("input_sample_2.txt")

31-element Array{Int64,1}:
 28
 33
 18
 42
 31
 14
 46
 20
 48
 47
 24
 23
 49
  ⋮
 32
 25
 35
  8
 17
  7
  9
  4
  2
 34
 10
  3

In [4]:
input_puzzle = parse_input("input_puzzle.txt")

92-element Array{Int64,1}:
  76
  51
 117
  97
   7
  77
  63
  18
 137
  10
  23
  14
 130
   ⋮
  92
 102
  88
  95
 132
  47
  50
  15
  68
  84
 136
 103

In [28]:
sort_adapters(adapters) = [0; sort(adapters); maximum(adapters) + 3]

sort_adapters (generic function with 1 method)

In [29]:
"""
The jolt difference from 0 up to the device, along all the adapters in jolt order.
"""
jolt_difference_path(adapters) = diff(sort_adapters(adapters))

jolt_difference_path

## part 1

In [163]:
jolt_difference_lookup(diffs) = Dict(d => count(diffs .== d) for d in unique(diffs))

jolt_difference_lookup (generic function with 1 method)

### answer

In [164]:
function show_answer_report(input, ::Val{:part1})
    diffs = jolt_difference_path(input)
    @show @timev d = jolt_difference_lookup(diffs)
    @info "Answer found." answer = d[1] * d[3]
    return
end

show_answer_report (generic function with 2 methods)

In [165]:
show_answer_report(input_sample_1, Val(:part1))

  0.000005 seconds (14 allocations: 1.438 KiB)
elapsed time (ns): 4626
bytes allocated:   1472
pool allocs:       14
#= In[164]:3 =# @timev(d = jolt_difference_lookup(diffs)) = Dict(3 => 5,1 => 7)


┌ Info: Answer found.
│   answer = 35
└ @ Main In[164]:4


In [166]:
show_answer_report(input_sample_2, Val(:part1))

  0.000004 seconds (14 allocations: 1.438 KiB)
elapsed time (ns): 4280
bytes allocated:   1472
pool allocs:       14
#= In[164]:3 =# @timev(d = jolt_difference_lookup(diffs)) = Dict(3 => 10,1 => 22)


┌ Info: Answer found.
│   answer = 220
└ @ Main In[164]:4


In [167]:
show_answer_report(input_puzzle, Val(:part1))

  0.000005 seconds (14 allocations: 1.438 KiB)
elapsed time (ns): 4891
bytes allocated:   1472
pool allocs:       14
#= In[164]:3 =# @timev(d = jolt_difference_lookup(diffs)) = Dict(3 => 24,1 => 69)


┌ Info: Answer found.
│   answer = 1656
└ @ Main In[164]:4


## part 2

This part is effectively some sort of Tribonacci, which cannot be calculated by naive recursion. There are usually 2 ways to handle this, using memoization of the recursion function (because it is an easy tag on), or use dynamic programming if there is a way to convert it.

It happened that there is a 3rd way in this problem, using the fact that there are only at most 5 consecutive adapters with 1-diff in jolt at one time, and there are only 1-diff and 3-diff.

### solution 1

Recursion with memoization.

In [168]:
function arrangements(diffs, ::Val{:solution1})
    lookup = Dict(length(diffs) => 1)

    function arrangements(i)
        if !haskey(lookup, i)
            total_arrangement = 0
            total_diff = 0
            for k in 1:3
                (i - 1 + k > length(diffs)) && break
                total_diff += diffs[i - 1 + k]
                (total_diff ≤ 3) ? (total_arrangement += arrangements(i + k)) : break
            end
            lookup[i] = total_arrangement
        end
        return lookup[i]
    end
    
    return arrangements(1)
end

arrangements (generic function with 3 methods)

### solution 2

Dynamic programming.

Alternatively if we need memory to be $O(1)$, it can be done by only remembering the latest 3 found values in the lookup as they are the only one used at most in the next lookup.

In [169]:
function arrangements(diffs, ::Val{:solution2})
    lookup = zeros(Int, length(diffs))
    lookup[1] = 1
    
    for i in eachindex(diffs)
        total_diff = 0
        for k in 1:3
            (i - k ≤ 0) && break
            total_diff += diffs[i - k]
            (total_diff ≤ 3) ? (lookup[i] += lookup[i - k]) : break
        end
    end
    
    return lookup[end]
end

arrangements (generic function with 3 methods)

### solution 3

Using the fact that we can count the arrangement for each consecutive 1-diff, and then multiply them together. This is possible because 3-diff means both adapters at the boundary have to be used. Also there is no 2-diff.

In addition, we optionally use hardcoding because the puzzle has at most 5 consecutive 1-diff.

In [170]:
"""
Number of valid arrangements of `n` 1-diff (i.e. n + 1 adapters of consecutive jolt),
such that the 1st and the last will be taken.
This is effectively the tribonacci number of non-negative integer starting with (1, 1, 2).
Only work for n ∈ 0:4 as it is hardcoded.
"""
arragements_1diff(n) = [1, 1, 2, 4, 7][n + 1]

arragements_1diff

In [171]:
"""
Count the number of consecutive 1-diff in each partition in the diffs.
"""
function consecutive_1diff(diffs)
    result = [0]
    index = 1
    for d in diffs
        if d == 1
            result[index] += 1
        else
            push!(result, 0)
            index += 1
        end
    end
    return result
end

consecutive_1diff

In [172]:
function arrangements(diffs, ::Val{:solution3})
    return (
        diffs
        |> consecutive_1diff
        .|> arragements_1diff
        |> prod
    )
end

arrangements (generic function with 3 methods)

### answer

In [173]:
function show_answer_report(input, ::Val{:part2}, solution_val)
    diffs = jolt_difference_path(input)
    @info "Answer found." @timev answer=arrangements(diffs, solution_val)
    return
end

show_answer_report (generic function with 2 methods)

In [187]:
show_answer_report(input_sample_1, Val(:part2), Val(:solution1))

  0.000006 seconds (9 allocations: 2.016 KiB)
elapsed time (ns): 6326
bytes allocated:   2064
pool allocs:       9


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 8
└ @ Main In[173]:3


In [188]:
show_answer_report(input_sample_1, Val(:part2), Val(:solution2))

  0.000000 seconds (1 allocation: 176 bytes)
elapsed time (ns): 412
bytes allocated:   176
pool allocs:       1


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 8
└ @ Main In[173]:3


In [189]:
show_answer_report(input_sample_1, Val(:part2), Val(:solution3))

  0.000001 seconds (10 allocations: 1.094 KiB)
elapsed time (ns): 1438
bytes allocated:   1120
pool allocs:       10


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 8
└ @ Main In[173]:3


In [190]:
@time show_answer_report(input_sample_2, Val(:part2), Val(:solution1))

  0.000015 seconds (40 allocations: 2.500 KiB)
elapsed time (ns): 14951
bytes allocated:   2560
pool allocs:       40
  0.000728 seconds (415 allocations: 14.250 KiB)


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 19208
└ @ Main In[173]:3


In [191]:
@time show_answer_report(input_sample_2, Val(:part2), Val(:solution2))

  0.000001 seconds (1 allocation: 336 bytes)
elapsed time (ns): 675
bytes allocated:   336
pool allocs:       1
  0.000627 seconds (374 allocations: 11.812 KiB)


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 19208
└ @ Main In[173]:3


In [192]:
@time show_answer_report(input_sample_2, Val(:part2), Val(:solution3))

  0.000002 seconds (16 allocations: 1.906 KiB)
elapsed time (ns): 2434
bytes allocated:   1952
pool allocs:       16
  0.000639 seconds (391 allocations: 13.656 KiB)


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 19208
└ @ Main In[173]:3


In [196]:
@time show_answer_report(input_puzzle, Val(:part2), Val(:solution1))

  0.000026 seconds (283 allocations: 10.859 KiB)
elapsed time (ns): 25515
bytes allocated:   11120
pool allocs:       281
non-pool GC allocs:2
  0.000670 seconds (725 allocations: 25.656 KiB)


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 56693912375296
└ @ Main In[173]:3


In [194]:
@time show_answer_report(input_puzzle, Val(:part2), Val(:solution2))

  0.000001 seconds (1 allocation: 896 bytes)
elapsed time (ns): 1193
bytes allocated:   896
pool allocs:       1
  0.000578 seconds (377 allocations: 13.938 KiB)


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 56693912375296
└ @ Main In[173]:3


In [195]:
@time show_answer_report(input_puzzle, Val(:part2), Val(:solution3))

  0.000004 seconds (31 allocations: 4.031 KiB)
elapsed time (ns): 4287
bytes allocated:   4128
pool allocs:       31
  0.000619 seconds (407 allocations: 17.641 KiB)


┌ Info: Answer found.
│   #= In[173]:3 =# @timev answer = arrangements(diffs, solution_val) = 56693912375296
└ @ Main In[173]:3
